In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats


togo= pd.read_csv(r"C:\Users\mike\Documents\Kifiya_AIM\data\togo-dapaong_qc.csv", parse_dates=["Timestamp"])

print(togo.head(7))

print(togo.dtypes)

print(togo.info())

print(togo.describe())

print(togo.isna().sum())

# Drop column: 'Comments'
togo = togo.drop(columns=['Comments'])
# Sort by column: 'Timestamp' (ascending)
togo = togo.sort_values(['Timestamp'])


